In [1]:
import filepath

In [2]:
# TODO: a room with walls
# a camera that can fly
# from panda3d.core import PointLight, DirectionalLight
import filepath
from util.app import ContextShowBase
from util.log import Loggable
from direct.task import Task
import numpy as np
import torch
from torch import Tensor
from geom.basic import create_cube_node, create_sphere_node,uv_curve_surface, create_colored_cube_node
from panda3d.core import (
    Geom,
    GeomNode,
    GeomTriangles,
    GeomVertexData,
    GeomVertexFormat,
    GeomVertexWriter,
    GeomEnums,
    NodePath,
    PointLight,
    DirectionalLight,
    CardMaker,
    WindowProperties
)
import ursina
from direct.showbase import DirectObject
from game.game_object import GameObject
from game.controller import PlayerController

# controller
# player controller
# agent controller
from typing import Set, List, Dict,Callable
from datetime import datetime
from abc import ABC
from direct.showbase.ShowBase import ShowBase
from panda3d.core import PNMImage, Texture
from panda3d.core import PTAUchar
from panda3d.core import CardMaker
from panda3d.core import Point2
from panda3d.core import NodePath, Camera, PerspectiveLens

import numpy as np
# import gizeh as gz
import random

In [3]:
import sympy
import torch

In [4]:
class CameraController:
    from panda3d.core import (
        LVector3f,
        LQuaternionf
    )
    def __init__(self,camera:Camera):
        self.camera = camera
        self.move_step = .5
        self.turn_step = 5
    # FIXME: use dict
    
    def call_move_forward(self, task):
        self.move(task, self.move_step)
        
    def call_move_backward(self, task):
        self.move(task, -self.move_step)
        
    def call_move_left(self, task):
        new_pos = self.getPos() - self.getRight(self.ref) * self.move_step
        self.setPos(new_pos)
    
        
    def call_move_right(self, task):
        new_pos = self.getPos() + self.getRight(self.ref) * self.move_step
        self.setPos(new_pos)
        
    def call_move_down(self, task):
        self.move_vertically(task, -self.move_step)
        
    def call_move_up(self, task):
        self.move_vertically(task, self.move_step)
    
    def setRef(self, ref):
        self.log("---set ref---:{},{}".format(ref,type(ref)))
        self.ref = ref
        
    def move(self, task:Task, dist:float):
        # self.log("move {}".format(dist))
        new_pos = self.getPos() + self.getForward(self.ref) * dist
        self.setPos(new_pos)
        
    def getForward(self, ref) -> LVector3f:
        return self.getQuat(ref).getForward()
    
    def getRight(self, ref) -> LVector3f:
        return self.getQuat(ref).getRight()
    
    def move_vertically(self, task:Task, dist:float):
        newZ = self.getZ() + dist
        self.setZ(newZ)
        
    def rotate(self, task: Task, angle:float):
        new_H = self.getH() + angle
        self.setH(new_H)
        
    def setPos(self,*args):
        self.camera.setPos(*args)
        
    def setHpr(self, *args):
        self.camera.setHpr(*args)
        
    def setH(self, *args):
        self.camera.setH(*args)
        
    def setP(self, *args):
        self.camera.setP(*args)
        
    def setR(self, *args):
        self.camera.setR(*args)
        
    def setZ(self, *args):
        self.camera.setZ(*args)
    
    def getQuat(self, *args):
        return self.camera.getQuat(*args)
    
    def getPos(self, *args):
        return self.camera.getPos(*args)
    
    def getZ(self, *args):
        return self.camera.getZ(*args)
    
    def toggle_default(self, task):
        pass
    
class PlayerCamController(CameraController, PlayerController):
    def __init__(self, camera): # TODO: input keymap dict
        CameraController.__init__(self, camera) # TODO
        PlayerController.__init__(self)
        # from panda3d.core import KeyboardButton
        self.key_maps = {
            'w': self.call_move_forward,
            's': self.call_move_backward,
            'a': self.call_move_left,
            'd': self.call_move_right,
            'lshift': self.call_move_up,
            'space':self.call_move_down
        }


In [5]:
from util.texture import (
    create_grey_checkerboard, 
    create_color_checkerboard,
    np2texture
)

In [6]:
def make_wall_texture(
    w, h,
    square_size:float=1,
    res:int=8,
    color1 = [0,0,0],
    color2 = [255,255,255],
    name = None
) -> Texture:
    w_img = int(w*res)
    h_img = int(h*res)
    square_size_img = int(square_size*res)
    checkerboard_arr = create_color_checkerboard(
        size=(h_img,w_img),
        square_size=square_size_img,
        color1=color1, color2=color2
    )[:,:,::-1]
    t = np2texture(
        arr=checkerboard_arr,
        format_=Texture.F_rgb,
        name=name
    )
    return t
    

In [7]:
from panda3d.bullet import BulletWorld
class PhyscRoom(ContextShowBase):
    def __init__(self, xb:int, yb:int , zb:int):
        super().__init__()
        self.name = "Physics Room"
        self.bullet_world = BulletWorld()
        self.paused = True
        # for controlledShowBase
        self.is_cursor_in_game:bool = True
        self.cam_controller = PlayerCamController(self.camera)
        self.cam_controller.setRef(self.render) ## FIXME: autoset
        
        resolution = 8 # 8pxs a grid
        tex_top = make_wall_texture(
            yb*2,xb*2,square_size=5,
            res=resolution,
            color1 = [135,206,235] # celestial 
        )
        tex_bot = make_wall_texture(
            yb*2,xb*2,square_size=5,
            res=resolution,
            color1 = [227,189,101], # mustard
            color2 = [34,139,34] # grass
        )
        tex_xz = make_wall_texture(
            zb*2,xb*2,square_size=5,
            res=resolution,
            color1= [255,187,255] # plum
        )
        tex_yz = make_wall_texture(
            zb*2,yb*2,square_size=5,
            res=resolution,
            color1 = [0,173,131], # Pantone,
        )        
        
        # maka a room
        cardxy = CardMaker('wallxy')
        cardxy.setFrame(-xb,xb,-yb,yb)
        # FIXME
        cardxy.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))  # ul
        wall_xy_top = self.render.attach_new_node(cardxy.generate())
        wall_xy_top.set_pos(0,0,zb)
        wall_xy_top.setP(90)
        wall_xy_top.set_texture(tex_top)
        wall_xy_bot = self.render.attach_new_node(cardxy.generate())
        wall_xy_bot.set_pos(0,0,-zb)
        wall_xy_bot.setP(-90)
        wall_xy_bot.set_texture(tex_bot)
        cardxz = CardMaker('wallxz')
        cardxz.setFrame(-xb,xb,-zb,zb)
        cardxz.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))  
        wall_xz_front = self.render.attach_new_node(cardxz.generate())
        wall_xz_front.set_pos(0,yb,0)
        wall_xz_front.set_texture(tex_xz)
        wall_xz_back = self.render.attach_new_node(cardxz.generate())
        wall_xz_back.set_pos(0,-yb,0)
        wall_xz_back.setH(-180)
        wall_xz_back.set_texture(tex_xz)
        cardyz = CardMaker('wallyz')
        cardyz.setFrame(-yb,yb,-zb,zb)
        cardyz.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))  
        wall_yz_left = self.render.attach_new_node(cardyz.generate())
        wall_yz_left.set_pos(-xb,0,0)
        wall_yz_left.setH(90)
        wall_yz_left.set_texture(tex_yz)
        wall_yz_right = self.render.attach_new_node(cardyz.generate())
        wall_yz_right.set_pos(xb,0,0)
        wall_yz_right.setH(-90)
        wall_yz_right.set_texture(tex_yz)
        
        sphere_o = create_sphere_node("o", 12, 12)
        self.render.attachNewNode(sphere_o)
        node_path_sphere_o = NodePath(sphere_o)
        node_path_sphere_o.setColor((1,1,1,1))
        node_path_sphere_o.setPos(0,0,0)
        sphere_x = create_sphere_node("x", 12, 12)
        self.render.attachNewNode(sphere_x)
        node_path_sphere_x = NodePath(sphere_x)
        node_path_sphere_x.setColor((1,0,0,1))
        node_path_sphere_x.setPos(1,0,0)
        sphere_y = create_sphere_node("y", 12, 12)
        self.render.attachNewNode(sphere_y)
        node_path_sphere_y = NodePath(sphere_y)
        node_path_sphere_y.setColor((0,1,0,1))
        node_path_sphere_y.setPos(0,1,0)
        sphere_z = create_sphere_node("z", 12, 12)
        self.render.attachNewNode(sphere_z)
        node_path_sphere_z = NodePath(sphere_z)
        node_path_sphere_z.setColor((0,0,1,1))
        node_path_sphere_z.setPos(0,0,1)
        node_path_sphere_o.setScale(0.2)
        node_path_sphere_x.setScale(0.2)
        node_path_sphere_y.setScale(0.2)
        node_path_sphere_z.setScale(0.2)
              
        
       
        # control ------------
        self.buttonThrowers[0].node().setButtonDownEvent('button')
        self.buttonThrowers[0].node().setButtonUpEvent('button-up')
        self.accept("space", lambda: print(self.camera.get_pos()))
        self.accept("escape", self.cursor_out)
        self.accept("b", self.cursor_in) # FIXME
        
        self.cursor_in()
        # 每帧更新摄像机
        self.default_cam_pos = (0, -10, 1)
        self.camera.setPos(*self.default_cam_pos)
        self.accept('z', self.toggle_camera) 
        self.accept('p', self.pause_switch)
        self.taskMgr.add(self.update_camera, "update_camera_task")
        # self.taskMgr.add(self.spin_card, "spin_card_task")
        self.taskMgr.add(self.cam_controller.update, "update_controller")
        self.taskMgr.add(self.update, 'updateWorld')
        # 保存鼠标的初始位置
        self.prev_mouse_x = 0
        self.prev_mouse_y = 0
        
    def cursor_in(self):
        # center the mouse
        self.center_mouse()
        # disable default mouse control 
        self.disable_mouse()
        # hide mouse cursor
        props = WindowProperties()
        props.setCursorHidden(True)
        self.win.requestProperties(props)
        # set state of the mouse, 
        # which controls whether camera updates
        self.is_cursor_in_game = True
        
    def cursor_out(self):
        # enable default mouse control
        self.enable_mouse()
        # show mouse cursor
        props = WindowProperties()
        props.setCursorHidden(False)
        self.win.requestProperties(props)
        # set state of the mouse, 
        # which controls whether camera updates
        self.is_cursor_in_game = False

    def center_mouse(self):
        """将鼠标指针重置到窗口的中心"""
        window_center_x = self.win.getXSize() // 2
        window_center_y = self.win.getYSize() // 2
        self.win.movePointer(0, window_center_x, window_center_y)
        self.prev_mouse_x = window_center_x
        self.prev_mouse_y = window_center_y
        
    def toggle_camera(self):
        self.camera.setPos(*self.default_cam_pos)
        self.camera.setHpr(0,0,0)

    def update_camera(self, task):
        """每帧更新摄像机的方向，使其跟随鼠标的移动"""
        if self.mouseWatcherNode.hasMouse() and self.is_cursor_in_game:
            # 获取鼠标的位置（归一化的 -1 到 1 范围内）
            mouse_x = self.win.getPointer(0).getX()
            mouse_y = self.win.getPointer(0).getY()

            # 计算鼠标移动的增量
            delta_x = (mouse_x - self.prev_mouse_x)
            delta_y = mouse_y - self.prev_mouse_y

            # 调整摄像机的水平旋转和俯仰角度
            camera_h = self.camera.getH() - delta_x * 0.1
            camera_p = self.camera.getP() - delta_y * 0.1

            # 设置新的摄像机角度
            self.camera.setH(camera_h)
            self.camera.setP(camera_p)

            # 将鼠标指针重置到窗口的中心
            self.center_mouse()
        return task.cont
  
    def toggle_fullscreen(self):
        props = WindowProperties()
        props.setFullscreen(not self.win.isFullscreen())
        self.win.requestProperties(props)
        
    def pause_switch(self):
        self.paused = not self.paused
        self.log("paused:{}".format(self.paused))
        return self.paused
        
    def update(self, task):  # FIXME: decorator
        if not self.paused:
            dt = globalClock.get_dt()
            self.bullet_world.do_physics(dt)
        return task.cont       

In [8]:
import builtins
import traceback
try:
    with PhyscRoom(25,25,25) as app:
        app.run()
except Exception as e:
    print(e)
    print(traceback.format_exc())
finally:
    if hasattr(builtins, 'base'):
        builtins.base.destroy()      

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


---set ref---:render,<class 'panda3d.core.NodePath'>
---Physics Room run(), at 2024-08-27 08:00:27.996257---
---Physics Room destroy at 2024-08-27 08:00:31.150789---
---Physics Room destroyed at 2024-08-27 08:00:31.151015, exit---


SystemExit: 

/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
from panda3d.core import Point3, LVector3f
from panda3d.bullet import (
    BulletWorld, 
    BulletBoxShape, 
    BulletPlaneShape,
    BulletRigidBodyNode, BulletSphereShape
)
import sympy as sp
from sympy.physics.units import (
    kilometer, meter,centimeter,
    gram, kilogram, tonne, 
    newton, second
)
from typing import List
from torch.nn import functional as F

In [11]:
# GameObject
#   .totol_mass

tmoon = Texture()
tmoon.read("res/moon.jpeg")
tmoon.setWrapU(Texture.WM_repeat)
tmoon.setWrapV(Texture.WM_repeat)

G_val = 6.67430*1e-11 * newton * meter**2 / (kilogram**2)
def getG(unit,G_val=G_val):
    G = sp.symbols('G')
    m1, m2 = sp.symbols('m1 m2')
    r = sp.symbols('r')
    M1 = m1 * unit["mass"]
    M2 = m2 * unit["mass"]
    R = r * unit["length"]
    F = G*(M1*M2)/R**2
    f = F/unit["force"]
    G_game = f.simplify().subs({
        m1:1,
        m2:1,
        r:1,
        G:G_val
    })
    return G_game
def safe_reciprocal(x):
    return torch.where(x==0, 0*x, torch.reciprocal(x))

one = sp.Number(1)

class MassedBall(): # not yet inherent GameObject
    def __init__(
        self,
        name,
        radius,
        mass,
        units:dict
        # lat_res,
        # lon_res,
        # geom_type
        # TODO: texture
    ):
        # super().__init__(name)
        self.worlds = []
        self.radius = radius
        self.mass = mass
        self.units = units
        # print(radius / units["length"])
        self.game_radius:float = (radius / units["length"]).simplify().evalf()
        self.game_mass: float = (mass / units["mass"]).simplify().evalf()
        # create geom
        self.geom_node = create_sphere_node(name, lat_res=12,lon_res=12)
        self.geom_np = NodePath(self.geom_node)
        # print(self.game_radius)
        self.geom_np.setScale(self.game_radius)
        # create rigid body
        self.rigid_body_node = BulletRigidBodyNode("SphrRg."+str(name))
        sphere_shape = BulletSphereShape(self.game_radius)
        self.rigid_body_node.add_shape(sphere_shape)
        self.rigid_body_node.set_mass(self.game_mass)
        self.rigid_body_np = NodePath(self.rigid_body_node)
        # bind geom to rigid body
        # self.rigid_body_np.attachNewNode(self.geom_node)
        self.geom_np.reparent_to(self.rigid_body_np)
        # bind self to rigid body
        # self.reparent_to(self.rigid)
        # self.geom_np.reparentTo(self)
        # todo: add collison
        
    
    def toBulletWorld(self, world:BulletWorld):
        self.worlds.append(world)
        world.attach_rigid_body(self.rigid_body_node)
        # if there is contraint, attach constraint
        
    def set_texture(self, t):
        self.geom_np.set_texture(t)
        
    def set_tex_scale(self, t):
        self.geom_np.set_tex_scale(t)
        
    def setPos(self, *args):
        # super().setPos(self,*args)
        self.rigid_body_np.setPos(*args)
    
    @property
    def mainPath(self):
        return self.rigid_body_np
        
    def getPos(self, other=None):
        if other is None:
            return self.rigid_body_np.getPos()
        elif isinstance(other, NodePath): 
            return self.rigid_body_np.getPos(other)
        else:
            return self.rigid_body_np.getPos(other.mainPath)
        
    def reparentTo(self, *args):
        self.rigid_body_np.reparentTo(*args)
    def reparent_to(self, *args):
        self.rigid_body_np.reparent_to(*args)
    
    def setZ(self,*args):
        self.rigid_body_np.setZ(*args)
        
    def apply_force(self, force, pos):
        # print(force, pos)
        self.rigid_body_node.apply_force(LVector3f(*force), LVector3f(*pos))

def autocomplete_units(unit:dict):
    force_unit = (
        unit["mass"] * unit["length"] / unit["time"] ** 2
    )/(kilogram * meter / second**2).simplify()
    unit["force"] = force_unit * newton

class PhysicsRoomBalls(PhyscRoom):
    @property
    def masses(self):
        if hasattr(self, "_masses"):
            return self._masses
        else:
            self._masses =  self.get_mass(self.gravitational_bodies)
            return self._masses
    def __init__(self, xb:int, yb:int , zb:int):
        super().__init__(xb,yb,zb)
        # objects
        
        self.bullet_world.setGravity((0,0,0)) # no global gravity
        self.unit = {
            "mass" : tonne,
            "length" : 100*meter,
            "time": 1 * second, 
            # "force" : sp.Number(1e3) * newton
        }
        autocomplete_units(self.unit)
        # self.G_game = getG(self.unit, G_val=G_val)
        self.G_game = 0.001
        self.G_game_tensor = torch.Tensor([self.G_game])
        
        self.planet1 = MassedBall(
            name="planet1",
            radius=100*meter,
            mass=1e6*tonne,
            units=self.unit
        )
        self.planet1.reparentTo(self.render)
        
        
        self.planet2 = MassedBall(
            name="planet2",
            radius=100*meter,
            mass=2e6*tonne,
            units=self.unit
        )
        self.planet2.reparentTo(self.render)
        self.planet2.toBulletWorld(self.bullet_world)
        
        
        self.planet1.set_texture(tmoon)
        self.planet2.set_texture(tmoon)
        self.planet2.setZ(4)
        self.planet1.setZ(-4)
        self.planet1.toBulletWorld(self.bullet_world)
        self.gravitational_bodies = [self.planet1, self.planet2]
        
        print(self.masses)
        dist = self.get_node_dist(self.gravitational_bodies)
        gravity = self.cal_gravity(self.masses, dist)
        print(gravity)
        
    
    def apply_gravitational_force(self, task):
        # N by N matrix, upper triaglar
        gravity = self.cal_gravity(
            self.masses,
            self.get_node_dist(self.gravitational_bodies),
            sumup=True
        )
        self.batch_apply_force(
            self.gravitational_bodies,
            gravity,
            self.get_node_poses(self.gravitational_bodies)
        )
        
        
    
    @staticmethod
    def get_node_poses(node_paths:List[NodePath]) -> Tensor:
        pos = torch.Tensor([
            n.getPos()
            for n in node_paths
        ])
        return pos
    
    @staticmethod
    def get_node_dist(node_paths:List[NodePath]) -> Tensor:
        nr_nodes = len(node_paths)
        # FIXME: use vector distance
        # dist ij
        return torch.Tensor([
            [
                node_paths[j].getPos(node_paths[i])
                # if i<j else (0,0,0)
                for j in range(nr_nodes)
            ]
            for i in range(nr_nodes)
        ])
    
    @staticmethod
    def get_mass(objs:List[MassedBall]) -> torch.Tensor:
        return torch.Tensor([
            o.game_mass for o in objs
        ])
    
    def cal_gravity(self, mass:torch.Tensor, dist:torch.Tensor, sumup=False) -> torch.Tensor:
        dist_sq = torch.einsum(
            "mnd -> mn",
            dist ** 2
        )
        dist_sq_rcpr = safe_reciprocal(dist_sq)
        gravity_scalar = torch.einsum(
            "o, n, mn, m -> mn",
            self.G_game_tensor, 
            mass, dist_sq_rcpr, mass
        )
        gravity_vector = torch.einsum(
            "mn, mnd -> mnd",
            gravity_scalar, 
            F.normalize(dist, dim=2, p=2)
        )
        if sumup:
            return torch.einsum(
                "mnd -> md",
                gravity_vector
            )
        return gravity_vector
        
    @staticmethod
    def batch_apply_force(bodies, force, pos):
        for i in range(len(bodies)):
            bodies[i].apply_force(
                force[i],
                pos[i]
            )
            
    def update(self, task):
        if not self.paused:
            self.apply_gravitational_force(task)
        return super().update(task)
    


    
    
        
import builtins
import traceback
try:
    with PhysicsRoomBalls(25,25,25) as app:
        app.run()
except Exception as e:
    print(e)
    print(traceback.format_exc())
finally:
    if hasattr(builtins, 'base'):
        builtins.base.destroy()             

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


---set ref---:render,<class 'panda3d.core.NodePath'>
tensor([1000000., 2000000.])
tensor([[[        0.,         0.,         0.],
         [        0.,         0.,  31250002.]],

        [[        0.,         0., -31250002.],
         [        0.,         0.,         0.]]])
---Physics Room run(), at 2024-08-27 12:45:34.937352---
paused:False
---Physics Room destroy at 2024-08-27 12:45:46.634153---
---Physics Room destroyed at 2024-08-27 12:45:46.634629, exit---


SystemExit: 

In [22]:
class PhysicsRoomPlane(PhyscRoom):
    def __init__(self, xb:int, yb:int , zb:int):
        super().__init__(xb,yb,zb)
        # objects
        
        self.bullet_world = BulletWorld()
        # self.bullet_world.setGravity((0,0,-9.81)) # no global gravity
        self.groundNode = BulletRigidBodyNode('ground')
        ground = BulletPlaneShape((0,0,1),1)
        self.groundNode.addShape(ground)
        self.groundNp = self.render.attachNewNode(self.groundNode)
        self.bullet_world.attach_rigid_body(self.groundNode)
        self.groundNp.setPos(0,0,-zb)
        
        self.sphereNode = BulletRigidBodyNode('sphere')
        sphere_rigid = BulletSphereShape(1)
        self.sphereNode.setMass(1)
        self.sphereNode.addShape(sphere_rigid)
        # self.sphereNp = self.render.attachNewNode(self.sphereNode)
        self.sphereNp = NodePath(self.sphereNode)
        self.sphereNp.reparentTo(self.render)
        sphere_geom = create_sphere_node('sphere01',lat_res=6,lon_res=6)
        sphere_geom_np = NodePath(sphere_geom)
        sphere_geom_np.reparentTo(self.sphereNp)
        sphere_geom_np.setPos(0,0,2)
        self.bullet_world.attach_rigid_body(self.sphereNode)
        sphere_geom_np.setTexture(tmoon)
        # you cannot use non physical parent nodepath to
        # set rigid pos
        # self.new_np = NodePath("hey")
        # self.sphereNp.reparent_to(self.new_np)
        # self.new_np.reparent_to(self.render)
        # self.sphereNp.setPos(0,0,2)
        # self.new_np.setPos(0,0,2)
        
    
        
import builtins
import traceback
try:
    with PhysicsRoomPlane(25,25,25) as app:
        app.run()
except Exception as e:
    print(e)
    print(traceback.format_exc())
finally:
    if hasattr(builtins, 'base'):
        builtins.base.destroy()             

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


---set ref---:render,<class 'panda3d.core.NodePath'>
---Physics Room run(), at 2024-08-27 08:38:56.303268---
---Physics Room destroy at 2024-08-27 08:39:00.361074---
---Physics Room destroyed at 2024-08-27 08:39:00.361552, exit---


SystemExit: 

In [105]:
autocomplete_units(unit)
# unit

In [ ]:
# model
# either reparent to some rigid body
# or a node, the position of node updates according to a function

# geom nodes categorized into groups
# each group renders one geom in certain condition
# rendership do not change each frame
# change it when certain criteria met

In [31]:
# calculate gravity

In [33]:

F = G*(m1*m2)/r**2
F

G*m1*m2/r**2

In [34]:
unit = {
            "mass" : tonne,
            "length" : 100*meter,
            "time": 1 * second, 
            # "force" : sp.Number(1e3) * newton
        }
autocomplete_units(unit)

In [35]:
unit

{'mass': tonne,
 'length': 100*meter,
 'time': second,
 'force': 100*newton*tonne/kilogram}

In [42]:
G_val = 6.67430*1e-11 * newton * meter**2 / (kilogram**2)
M1 = m1 * unit["mass"]
M2 = m2 * unit["mass"]
R = r * unit["length"]
F = G*(M1*M2)/R**2
f = F/unit["force"]
G_game = f.simplify().subs({
    m1:1,
    m2:1,
    r:1,
    G:G_val
})

In [43]:
G_game

6.67430000000000e-14

In [78]:
x = torch.Tensor([[1,0],[0,0]])*torch.Tensor([one])


In [72]:
type(x.to(dtype=float)[0].item())

float

In [73]:
type(torch.reciprocal(x)[0].item())

float

In [76]:
torch.Tensor([one])

tensor([1.])

In [70]:
one = sp.Number(1)

In [80]:
F.normalize(x, dim=-1,p=2)

tensor([[1., 0.],
        [0., 0.]])

In [88]:
x.numpy()

array([[1., 0.],
       [0., 0.]], dtype=float32)

In [90]:
y = torch.Tensor([1,2,3])

In [93]:
LVector3f(*y)

LVector3f(1, 2, 3)